# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 21096902


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/30 [00:00<?, ?it/s]

Rendering models:   7%|▋         | 2/30 [00:06<01:37,  3.47s/it]

Rendering models:  10%|█         | 3/30 [00:07<01:08,  2.53s/it]

Rendering models:  13%|█▎        | 4/30 [00:07<00:47,  1.83s/it]

Rendering models:  17%|█▋        | 5/30 [00:07<00:34,  1.39s/it]

...

Rendering models:  87%|████████▋ | 26/30 [00:12<00:00,  4.16it/s]

Rendering models:  90%|█████████ | 27/30 [00:12<00:00,  4.02it/s]

Rendering models:  93%|█████████▎| 28/30 [00:13<00:00,  4.49it/s]

Rendering models:  97%|█████████▋| 29/30 [00:13<00:00,  4.72it/s]

Jnursssmith5263                       0.002291
ethanth                               0.000418
not-logged-in-553c4a543f16d42b2154    0.018657
sorenjensen01                         0.002208
24declanfm                            0.085924
jnarayanbvg                           0.003954
awright5                              0.001705
severing_ties_with_dema               0.000446
GensC                                 0.063514
pollux2728                            0.004713
FredHendar                            0.000385
tingard                               0.000300
crush202020                           0.001187
jzeiszler                             0.000731
not-logged-in-214814e52004b42491a4    0.075285
RaulE                                 0.000433
Lavadude                              0.025707
BrowardDaniel                         0.001500
equidad1                              0.003929
LeusaneLordelo                        0.000462
Denny2507                             0.010262
Fla          

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())